# **Model Experimentation and Criteria for Evaluation**
* [4.1 Model Choice](#4.1)
* [4.2 Fitting the Model](#4.2)
* [4.3 Making Predictions](#4.3)
* [4.4. Making predictions](#4.4)

In [ ]:
"""
- tratar da introdução
- fazer markdowns

"""

<a id="4.1"></a>     


## **4.1. Model Choice**

o nb 9 tem de ser independente

abrir o dataset original
dropar as features que tivermos escolhido
tudo o resto que fizemos aos dados na parte do preprocessing


aplicar o scaler

treinar o modelo final (que escolhemos no nb4)

Não precisamos de fazer o split, treinamos com os dados todos

FAZER UM MARKDOWN A EXPLICAR A INDEPENÊNCIA DO NB 9 E APLICARMOS TUDO O QUE FIZEMOS NOS OUTROS NOTEBOOKS

"APLICANDO O QUE FOI FEITO NO NOTEBOOK X... tal tal tal

In [1]:
"""
Importing the necessary libraries
"""


import pandas as pd
import numpy as np
import pickle, os
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignore ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder

In [2]:
dataset_original = pd.read_csv('Nata_Files/learn.csv', index_col = 0)
dataset_original

,ambient_humidity,baking_duration,cooling_period,cream_fat_content,egg_temperature,egg_yolk_count,final_temperature,lemon_zest_ph,notes_baker,origin,oven_temperature,pastry_type,preheating_time,salt_ratio,sugar_content,vanilla_extract,quality_class
id,,,,,,,,,,,,,,,,,
1,54.0,24.0,26.0,100.4,52.0,11.0,309.0,3.2,NaN,Lisboa,NaN,Pastel Nata,207.0,42.74,22.8,5.7,KO
2,66.0,37.0,34.0,98.0,46.0,10.0,317.0,3.3,NaN,Lisboa,306.0,NaN,245.0,41.73,11.6,4.0,KO
3,41.0,30.0,19.0,99.3,53.0,10.0,130.0,3.4,NaN,Porto,121.0,NaN,186.0,75.10,20.3,7.5,OK
4,62.0,24.0,48.0,98.0,115.0,9.0,354.0,3.3,NaN,Lisboa,357.0,Pastel de Nata,186.0,46.41,73.3,4.2,OK
5,55.0,21.0,34.0,100.1,48.0,9.0,211.0,3.0,NaN,Lisboa,202.0,Pastel de nata,218.0,56.52,80.1,6.0,KO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5196,60.0,18.0,35.0,96.0,72.0,11.0,215.0,3.3,NaN,Lisboa,222.0,NaN,177.0,34.42,58.9,5.7,OK
5197,61.0,25.0,40.0,96.4,99.0,9.0,367.0,3.2,NaN,Lisboa,366.0,Pastel De Nata,224.0,46.18,141.4,6.5,KO
5198,69.0,18.0,36.0,97.7,90.0,11.0,206.0,3.6,NaN,Lisboa,203.0,Pastel de nata,158.0,28.46,10.0,6.0,OK


In [3]:
'''Loading the prediction dataset'''
predict_data = pd.read_csv('Nata_Files/predict.csv', index_col = 0)
display(predict_data.shape)
predict_data

(1300, 16)

,ambient_humidity,baking_duration,cooling_period,cream_fat_content,egg_temperature,egg_yolk_count,final_temperature,lemon_zest_ph,notes_baker,origin,oven_temperature,pastry_type,preheating_time,salt_ratio,sugar_content,vanilla_extract
id,,,,,,,,,,,,,,,,
5201,79.0,22,40,98.6,79.0,9.0,259.0,3.2,NaN,Lisboa,268.0,NaN,208.0,49.63,182.6,4.0
5202,49.0,26,32,101.9,105.0,9.0,287.0,3.2,NaN,Lisboa,287.0,Pastel de nata,189.0,182.54,76.2,4.8
5203,80.0,28,24,96.6,20.0,10.0,64.0,3.4,NaN,Porto,74.0,Pastel Nata,201.0,100.41,23.5,6.1
5204,74.0,21,37,97.2,81.0,9.0,314.0,3.0,NaN,Lisboa,317.0,NaN,220.0,46.66,143.2,4.9
5205,41.0,19,41,97.3,104.0,10.0,246.0,3.2,NaN,Lisboa,243.0,Pastel Nata,191.0,39.45,143.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6496,73.0,24,23,98.9,58.0,10.0,208.0,3.1,NaN,Lisboa,200.0,Pastel De Nata,189.0,37.80,142.6,4.0
6497,66.0,21,34,100.6,62.0,10.0,277.0,3.1,NaN,Lisboa,264.0,Pastel de nata,235.0,33.78,102.2,4.8
6498,41.0,49,23,98.9,20.0,11.0,57.0,3.4,NaN,Porto,69.0,Pastel de nata,243.0,86.73,21.3,7.4


In [4]:
dataset_original.drop(columns=['notes_baker','pastry_type'], inplace=True)


In [5]:
dataset_original.dropna(subset=['quality_class'], inplace=True)

dataset_original.isna().sum()

ambient_humidity      17
baking_duration        0
cooling_period         0
cream_fat_content     23
egg_temperature       23
egg_yolk_count        23
final_temperature     24
lemon_zest_ph         25
origin               160
oven_temperature      20
preheating_time       18
salt_ratio            12
sugar_content         21
vanilla_extract       17
quality_class          0
dtype: int64

In [6]:
#A COLUNA DO LISBOA E PORTO EM 0 E 1 OU NÃO... PÔR AQUI

In [7]:
dataset_original.loc[:, 'egg_yolk_count'] = dataset_original['egg_yolk_count'].astype('Int64')

C:\Users\tiago\AppData\Local\Temp\ipykernel_24636\746875755.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<IntegerArray>
[11, 10, 10,  9,  9,  9,  9, 13,  9, 11,
 ...
  9, 11, 11, 13, 12, 11,  9, 11,  9, 10]
Length: 5199, dtype: Int64' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataset_original.loc[:, 'egg_yolk_count'] = dataset_original['egg_yolk_count'].astype('Int64')


In [8]:
dataset_original['quality_class'].replace({'OK': 1, 'KO': 0}, inplace=True)


C:\Users\tiago\AppData\Local\Temp\ipykernel_24636\1450073272.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset_original['quality_class'].replace({'OK': 1, 'KO': 0}, inplace=True)
C:\Users\tiago\AppData\Local\Temp\ipykernel_24636\1450073272.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


In [9]:
''' 
We will use the same logic applied to dataset_original to clean the prediction dataset.
Firstly, we will drop the unneeded columns and then get rid of inconsistencies in the 'origin' column
'''
predict_data = predict_data.drop(columns=[ 'notes_baker', 'pastry_type']) 
predict_data['origin'] = predict_data['origin'].str.strip().str.lower().str.title()
print(predict_data['origin'].unique()) #just to check if it worked

['Lisboa' 'Porto' nan]


In [10]:
# 1st Constraint: Percentages (0 to 100)
impossible_humidityL = dataset_original[(dataset_original['ambient_humidity'] > 100) | (dataset_original['ambient_humidity'] < 0)]
impossible_fatL = dataset_original[(dataset_original['cream_fat_content'] > 100) | (dataset_original['cream_fat_content'] < 0)]

impossible_humidityP = predict_data[(predict_data['ambient_humidity'] > 100) | (predict_data['ambient_humidity'] < 0)]
impossible_fatP = predict_data[(predict_data['cream_fat_content'] > 100) | (predict_data['cream_fat_content'] < 0)]

# 2nd Constraint: pH Scale (0 to 14)
impossible_phL = dataset_original[(dataset_original['lemon_zest_ph'] > 14) | (dataset_original['lemon_zest_ph'] < 0)]
impossible_phP = predict_data[(predict_data['lemon_zest_ph'] > 14) | (predict_data['lemon_zest_ph'] < 0)]

In [11]:
dataset_original.loc[impossible_fatL.index, 'cream_fat_content'] = np.nan
predict_data.loc[impossible_fatP.index, 'cream_fat_content'] = np.nan

In [12]:
dataset_original['sugar_fat_ratio'] = dataset_original['sugar_content'] / (dataset_original['cream_fat_content'] + 1e-6)  # the small constant to avoid division by zero
dataset_original['baking_intensity'] = dataset_original['baking_duration'] * dataset_original['oven_temperature']

predict_data['sugar_fat_ratio'] = predict_data['sugar_content'] / (predict_data['cream_fat_content'] + 1e-6)
predict_data['baking_intensity'] = predict_data['baking_duration'] * predict_data['oven_temperature']

In [13]:
X_origin = dataset_original.drop('quality_class', axis = 1)
y_origin = dataset_original['quality_class']



In [14]:
numerical_features = X_origin.select_dtypes(include = np.number).columns.tolist()
categorical_features = X_origin.select_dtypes(exclude = np.number).columns.tolist()



num_features_origin = X_origin.select_dtypes(include = np.number).columns
cat_features_origin = X_origin.select_dtypes(exclude = np.number).columns

num_features_predict = predict_data.select_dtypes(include = np.number).columns
cat_features_predict = predict_data.select_dtypes(exclude = np.number).columns

dataset_original.dtypes

ambient_humidity     float64
baking_duration      float64
cooling_period       float64
cream_fat_content    float64
egg_temperature      float64
egg_yolk_count         Int64
final_temperature    float64
lemon_zest_ph        float64
origin                object
oven_temperature     float64
preheating_time      float64
salt_ratio           float64
sugar_content        float64
vanilla_extract      float64
quality_class          int64
sugar_fat_ratio      float64
baking_intensity     float64
dtype: object

In [15]:
print("Numerical Features:", num_features_origin)
print("numerical_features:", num_features_predict)

X_origin_num = X_origin[num_features_origin]
X_origin_cat = X_origin[cat_features_origin]

X_predict_num = predict_data[num_features_predict]
X_predict_cat = predict_data[cat_features_predict]

Numerical Features: Index(['ambient_humidity', 'baking_duration', 'cooling_period',
       'cream_fat_content', 'egg_temperature', 'egg_yolk_count',
       'final_temperature', 'lemon_zest_ph', 'oven_temperature',
       'preheating_time', 'salt_ratio', 'sugar_content', 'vanilla_extract',
       'sugar_fat_ratio', 'baking_intensity'],
      dtype='object')
numerical_features: Index(['ambient_humidity', 'baking_duration', 'cooling_period',
       'cream_fat_content', 'egg_temperature', 'egg_yolk_count',
       'final_temperature', 'lemon_zest_ph', 'oven_temperature',
       'preheating_time', 'salt_ratio', 'sugar_content', 'vanilla_extract',
       'sugar_fat_ratio', 'baking_intensity'],
      dtype='object')


In [16]:
imputer = SimpleImputer(strategy='median')
imputer.fit(X_origin_num)

,missing_values,nan
,strategy,'median'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False


In [17]:
X_origin_num = imputer.transform(X_origin_num)
X_predict_num = imputer.transform(X_predict_num)

In [18]:
#ENCODING PART!!

In [19]:
scaler = RobustScaler()
#Fitting the scaler on the training data
scaler.fit(X_origin_num)

#Transforming training data
X_origin_num_scaled = scaler.transform(X_origin_num)
X_orgin_num_scaled = pd.DataFrame(X_origin_num_scaled, columns = numerical_features, index = X_origin.index)

#Transforming prediction data
X_predict_num_scaled = scaler.transform(X_predict_num)
X_predict_num_scaled = pd.DataFrame(X_predict_num_scaled, columns = numerical_features, index = predict_data.index)



#PFF ALGUEM CONFIRME SE ISTO ESTÁ CERTO PLEASE

### Feature Selection

A PARTIR DAQUI FALTA PERCEBER A CENA DA FULL DATA SEM SPLIT OU SE FAZEMOS SPLIT EALTERAR AS FUNÇÕES (NOMES, ETC) VER COMO ESTÃO NAS AULAS E USAR ESSAS

In [ ]:
features_to_drop = ['ambient_humidity', 'sugar_fat_ratio', 'lemon_zest_ph']

X_origin_final = pd.concat([X_origin_num_scaled, X_origin_cat_encoded], axis=1)
X_predict_final = pd.concat([X_predict_num_scaled, X_predict_cat_encoded], axis = 1)

X_origin_final.drop(columns = features_to_drop, inplace= True)
X_predict_final.drop(columns = features_to_drop, inplace = True)

TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

Training the final model

In [21]:
"""
Training the final model with the best combination of parameters
"""

final_model = GradientBoostingClassifier(n_estimators=100, max_leaf_nodes=9, max_depth=6, loss='log_loss', learning_rate=0.1)
final_model.fit(X_origin_final, y_origin)

NameError: name 'X_origin_final' is not defined

In [ ]:

#PERCEBER ISTO!!!!!

kf = KFold(n_splits = 10,random_state = 42, shuffle = True)
rkf = RepeatedKFold(n_splits = 5, n_repeats = 3,random_state = 42)

def eval_model_clf(X, y, model):
    y_pred = model.predict(X)
    return accuracy_score(y, y_pred)
def run_model(X, y, model):
    return model.fit(X,y)

def avg_score_clf(method, X, y, model):
    score_train = []
    score_test = []
    for train_index, test_index in method.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        train_model = run_model(X_train, y_train, model)
        value_train = eval_model_clf(X_train, y_train, train_model)
        value_test = eval_model_clf(X_test, y_test, train_model)

        score_train.append(value_train)
        score_test.append(value_test)

    mean_train_score = np.mean(score_train)
    mean_test_score = np.mean(score_test)

    result_df = pd.DataFrame({'Train accuracy': [mean_train_score], 'Test accuracy': [mean_test_score]})
    result_df.index = [f'{str(model)} with {str(method)}']

    return result_df

In [ ]:
final_model_train_acc = eval_model_clf(X_origin_final, y_origin, final_model)
final_model_df = pd.DataFrame({'Train accuracy': [final_model_train_acc], 'Test accuracy': [final_model_test_acc]})
final_model_df.index = [f"{str(final_model)} with Simple Data Split"]

final_model_kf_df = avg_score_clf(kf, X_train_final, y_train,final_model)
final_model_rkf_df = avg_score_clf(rkf, X_train_final, y_train,final_model)

df_final_model = pd.concat([final_model_df, final_model_kf_df, final_model_rkf_df])

In [ ]:
def predict_and_results_metrics(final_model):
    predictions_train = final_model.predict(X_train_final)
    predictions_validation = final_model.predict(X_val_final)

    print('___________________________________________________________________________________________________________')
    print('                                                     TRAIN                                                 ')
    print('-----------------------------------------------------------------------------------------------------------')
    print(classification_report(y_train, predictions_train))
    print('Confusion Matrix:\n', confusion_matrix(y_train, predictions_train))

    print('___________________________________________________________________________________________________________')
    print('                                                VALIDATION                                                 ')
    print('-----------------------------------------------------------------------------------------------------------')
    print(classification_report(y_val, predictions_validation))
    print('Confusion Matrix:\n', confusion_matrix(y_val, predictions_validation))

In [ ]:
display(predict_and_results_metrics(final_model))

For Kaggle prediction

In [ ]:
X_full = pd.concat([X_train_final, X_val_final, X_test_final], axis=0)
y_full = pd.concat([y_train, y_val, y_test], axis=0)


final_model.fit(X_full, y_full)
# 3. Predict on the Kaggle data
final_predictions = final_model.predict(X_predict_final)

# 4. Save
submission = pd.DataFrame({'id': id_predict, 'Quality_class': final_predictions})
submission['Quality_class'] = submission['Quality_class'].map({0: 'KO', 1: 'OK'})
submission.to_csv('submission.csv', index=False)